In [1]:
from bs4 import BeautifulSoup
import requests, re, time

In [93]:
# vergadering = soup.find(id='broodtekst')

In [14]:
achternamen = []
partijen = []
teksten = []

In [15]:
def parse_item(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for spreekbeurt in soup.find_all(class_=['spreekbeurt', 'spreekbeurt eerst']):
        spreker = spreekbeurt.find(class_='spreker')
        achternaam = spreker.find(class_='achternaam').get_text()
        partij = spreker.find(class_='politiek')
        if not partij is None:
            partij = partij.get_text()

        tekst = spreekbeurt.find_all(class_=['alineagroep', 'motie'])
        tekst = ' '.join([alinea.get_text() for alinea in tekst])

        achternamen.append(achternaam)
        partijen.append(partij)
        teksten.append(tekst)

In [16]:
year = '2013-2014'

In [26]:
for i in range(1,1000): # !
    print(i)
    url = 'https://zoek.officielebekendmakingen.nl/rss/handelingen/TK/'+year+'/'+str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for br in soup.find_all('br'):
        br.replace_with('\n')
    urls = re.findall('<link/>(.*?)<category>', str(soup))
    if len(urls)==0:
        break
    for url in urls[1:]:
        parse_item(url)
    time.sleep(1)

49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


In [18]:
len(teksten)

29162

In [27]:
woorden = 0
for t in teksten:
    woorden += len(t)
print('Woorden: {}'.format(woorden))

Woorden: 43396964


In [28]:
import pandas as pd

In [29]:
df = pd.DataFrame([achternamen, partijen, teksten], index=['Naam', 'Partij', 'Uitspraak']).T

In [30]:
df.head()

,Naam,Partij,Uitspraak
0,voorzitter,None,\nIk deel aan de Kamer mee dat het volgende li...
1,voorzitter,None,\nIk geef het woord aan de minister van Financ...
2,Dijsselbloem,None,\nMevrouw de voorzitter. Het is een eer om hie...
3,voorzitter,None,\nIk zal er zuinig op zijn.\n \nIk dank de min...
4,voorzitter,None,"\nIk stel voor, als commissies bedoeld in arti..."


In [31]:
year

'2013-2014'

In [32]:
df.to_csv('uitspraken Tweede Kamer'+year+'.csv', index=False)